# Лабораторная работа 2
ЭН1-35 Кутбиддинов Т. Р. Полозов А. А. Вариант 13

#### Импортируем библиотеки

In [1]:
# !pip install pandapower

In [2]:
import pandapower as pp
import pandas as pd
import matplotlib.pyplot as plt
import math

#### Копируем конфигурацию сети из предыдущей лабораторной

In [3]:
# # Создание пустой сети
# net = pp.create_empty_network()

# # Создание узлов
# bus0 = pp.create_bus(net, name='system',  vn_kv=110)
# bus1 = pp.create_bus(net, name='load1',   vn_kv=110)
# bus2 = pp.create_bus(net, name='load2',   vn_kv=110)
# bus3 = pp.create_bus(net, name='load3',   vn_kv=110)
# bus4 = pp.create_bus(net2525, name='load4',   vn_kv=10)
# bus5 = pp.create_bus(net, name='load5',   vn_kv=10)
# bus6 = pp.create_bus(net, name='load6',   vn_kv=10)
# bus7 = pp.create_bus(net, name='load7',   vn_kv=10)
# bus8 = pp.create_bus(net, name='load8',   vn_kv=10)

# # Добавочные узлы
# bus08 = pp.create_bus(net, name='load08', vn_kv=110)
# bus35 = pp.create_bus(net, name='load35', vn_kv=110)
# bus34 = pp.create_bus(net, name='load34', vn_kv=110)

# # Подключение внешней энергосистемы
# pp.create_ext_grid(net, bus=bus0, vm_pu=1.02)

# # Создание нагрузок
# pp.create_load(net, bus=bus1, p_mw=12.0,  q_mvar=11.5)
# pp.create_load(net, bus=bus2, p_mw=23.0,  q_mvar=21.5)
# pp.create_load(net, bus=bus3, p_mw=27.0,  q_mvar=22.0)
# pp.create_load(net, bus=bus4, p_mw=5.0,   q_mvar=3.0)
# pp.create_load(net, bus=bus5, p_mw=3.0,   q_mvar=2.0)
# pp.create_load(net, bus=bus6, p_mw=4.0,   q_mvar=3.0)
# pp.create_load(net, bus=bus7, p_mw=2.5,   q_mvar=1.6)
# pp.create_load(net, bus=bus8, p_mw=4.0,   q_mvar=3.5)

# # Создание электрической линии
# line1 = pp.create_line_from_parameters(
#     net,
#     from_bus=bus0,
#     to_bus=bus08,
#     length_km=6.1,
#     r_ohm_per_km=0.46,
#     x_ohm_per_km=0.275,
#     max_i_ka=0.265,
#     c_nf_per_km=0
# )

# line2 = pp.create_line_from_parameters(
#     net,
#     from_bus=bus3,
#     to_bus=bus35,
#     length_km=9.6,
#     r_ohm_per_km=0.46,
#     x_ohm_per_km=0.275,
#     max_i_ka=0.265,
#     c_nf_per_km=0
# )

# line3 = pp.create_line_from_parameters(
#     net,
#     from_bus=bus3,
#     to_bus=bus34,
#     length_km=8.7,
#     r_ohm_per_km=0.46,
#     x_ohm_per_km=0.275,
#     max_i_ka=0.265,
#     c_nf_per_km=0
# )

# line4 = pp.create_line_from_parameters(
#     net,
#     from_bus=bus1,
#     to_bus=bus2,
#     length_km=1.4,
#     r_ohm_per_km=0.33,
#     x_ohm_per_km=0.234,
#     max_i_ka=0.33,
#     c_nf_per_km=0
# )

# line5 = pp.create_line_from_parameters(
#     net,
#     from_bus=bus8,
#     to_bus=bus7,
#     length_km=1.1,
#     r_ohm_per_km=0.12,
#     x_ohm_per_km=0.378,
#     max_i_ka=0.605,
#     c_nf_per_km=0
# )

# line6 = pp.create_line_from_parameters(
#     net,
#     from_bus=bus5,
#     to_bus=bus6,
#     length_km=1.3,
#     r_ohm_per_km=0.157,
#     x_ohm_per_km=0.377,
#     max_i_ka=0.51,
#     c_nf_per_km=0
# )

# line7 = pp.create_line_from_parameters(
#     net,
#     from_bus=bus0,
#     to_bus=bus1,
#     length_km=1.2,
#     r_ohm_per_km=0.33,
#     x_ohm_per_km=0.234,
#     max_i_ka=0.33,
#     c_nf_per_km=0
# )

# # Создание конденсаторных батарей
# pp.create_switch(
#     net,
#     bus=bus2,
#     element=bus3,
#     et="b",
#     closed=True
# )

# pp.create_switch(
#     net,
#     bus=bus4,
#     element=bus5,
#     et="b",
#     closed=True
# )

# pp.create_switch(
#     net,
#     bus=bus7,
#     element=bus6,
#     et="b",
#     closed=True
# )

# # Создание трансформаторов
# trafo1 = pp.create_transformer_from_parameters(
#     net,
#     hv_bus=bus08,
#     lv_bus=bus8,
#     sn_mva=32,
#     vn_hv_kv=110,
#     vn_lv_kv=10,
#     vk_percent=10.5,
#     i0_percent=0.28,
#     pfe_kw=160,
#     vkr_percent=(160 / 320)
# )

# trafo2 = pp.create_transformer_from_parameters(
#     net,
#     hv_bus=bus35,
#     lv_bus=bus5,
#     sn_mva=40,
#     vn_hv_kv=110,
#     vn_lv_kv=10,
#     vk_percent=11,
#     i0_percent=0.3,
#     pfe_kw=170,
#     vkr_percent=(170 / 400)
# )

# trafo3 = pp.create_transformer_from_parameters(
#     net,
#     hv_bus=bus34,
#     lv_bus=bus4,
#     sn_mva=40,
#     vn_hv_kv=110,
#     vn_lv_kv=10,
#     vk_percent=11,
#     i0_percent=0.3,
#     pfe_kw=170,
#     vkr_percent=(170 / 400)
# )

In [ ]:
# Объявим глобальные переменные (опционально, но хорошая практика — инициализировать их)
bus0 = bus1 = bus2 = bus3 = bus4 = bus5 = bus6 = bus7 = bus8 = None
bus08 = bus35 = bus34 = None

def create_network(net_name: str):
    """
    Создаёт и возвращает модель электрической сети с заданным
    именем.

    Сеть включает:
    - 12 узлов (9 основных + 3 дополнительных для трансформаторов),
    - внешнюю энергосистему (ext_grid),
    - 8 нагрузок,
    - 7 линий электропередачи с заданными параметрами,
    - 3 трансформатора 110/10 кВ,
    - 3 коммутационных выключателя (switches), соединяющих
      пары узлов.

    Параметры:
    ----------
    net_name : str
        Имя сети. Будет присвоено атрибуту `name` объекта
        pandapowerNet.
        Пример: "Industrial_Substation", "Test_Network_1".

    Возвращает:
    -----------
    net : pandapower.auxiliary.pandapowerNet
        Объект сети, готовый к дальнейшему анализу (например,
        power flow, short-circuit и т.д.).

    Инструкция к использованию:
    --------------------------
    1. Убедитесь, что установлен модуль `pandapower`:
       pip install pandapower

    2. Импортируйте функцию (если она находится в отдельном
       файле, например `network_builder.py`):
       from network_builder import create_network

    3. Вызовите функцию, передав желаемое имя сети:
       net = create_network("MyNetwork")

    4. Выполните расчёт установившегося режима (power flow):
       pp.runpp(net)

    5. Просмотрите результаты:
       print(net.res_bus)      # напряжения в узлах
       print(net.res_line)     # токи и потери в линиях
       print(net.res_trafo)    # загрузка трансформаторов

    Примечание:
    -----------
    Все параметры сети (нагрузки, длины линий, параметры
    трансформаторов и т.д.) жёстко заданы внутри функции и
    соответствуют типовой конфигурации, описанной в исходном
    коде. Для изменения топологии или параметров необходимо
    модифицировать тело функции.
    """
    # Создание пустой сети с указанным именем
    net = pp.create_empty_network(
        name=net_name
    )

    # Создание узлов
    bus0 = pp.create_bus(
        net=net,
        name='system',
        vn_kv=110
    )
    bus1 = pp.create_bus(
        net=net,
        name='load1',
        vn_kv=110
    )
    bus2 = pp.create_bus(
        net=net,
        name='load2',
        vn_kv=110
    )
    bus3 = pp.create_bus(
        net=net,
        name='load3',
        vn_kv=110
    )
    bus4 = pp.create_bus(
        net=net,
        name='load4',
        vn_kv=10
    )
    bus5 = pp.create_bus(
        net=net,
        name='load5',
        vn_kv=10
    )
    bus6 = pp.create_bus(
        net=net,
        name='load6',
        vn_kv=10
    )
    bus7 = pp.create_bus(
        net=net,
        name='load7',
        vn_kv=10
    )
    bus8 = pp.create_bus(
        net=net,
        name='load8',
        vn_kv=10
    )

    # Добавочные узлы
    bus08 = pp.create_bus(
        net=net,
        name='load08',
        vn_kv=110
    )
    bus35 = pp.create_bus(
        net=net,
        name='load35',
        vn_kv=110
    )
    bus34 = pp.create_bus(
        net=net,
        name='load34',
        vn_kv=110
    )

    # Подключение внешней энергосистемы
    pp.create_ext_grid(
        net=net,
        bus=bus0,
        vm_pu=1.02
    )

    # Создание нагрузок
    pp.create_load(
        net=net,
        bus=bus1,
        p_mw=12.0,
        q_mvar=11.5
    )
    pp.create_load(
        net=net,
        bus=bus2,
        p_mw=23.0,
        q_mvar=21.5
    )
    pp.create_load(
        net=net,
        bus=bus3,
        p_mw=27.0,
        q_mvar=22.0
    )
    pp.create_load(
        net=net,
        bus=bus4,
        p_mw=5.0,
        q_mvar=3.0
    )
    pp.create_load(
        net=net,
        bus=bus5,
        p_mw=3.0,
        q_mvar=2.0
    )
    pp.create_load(
        net=net,
        bus=bus6,
        p_mw=4.0,
        q_mvar=3.0
    )
    pp.create_load(
        net=net,
        bus=bus7,
        p_mw=2.5,
        q_mvar=1.6
    )
    pp.create_load(
        net=net,
        bus=bus8,
        p_mw=4.0,
        q_mvar=3.5
    )

    # Создание электрических линий
    pp.create_line_from_parameters(
        net=net,
        from_bus=bus0,
        to_bus=bus08,
        length_km=6.1,
        r_ohm_per_km=0.46,
        x_ohm_per_km=0.275,
        max_i_ka=0.265,
        c_nf_per_km=0
    )

    pp.create_line_from_parameters(
        net=net,
        from_bus=bus3,
        to_bus=bus35,
        length_km=9.6,
        r_ohm_per_km=0.46,
        x_ohm_per_km=0.275,
        max_i_ka=0.265,
        c_nf_per_km=0
    )

    pp.create_line_from_parameters(
        net=net,
        from_bus=bus3,
        to_bus=bus34,
        length_km=8.7,
        r_ohm_per_km=0.46,
        x_ohm_per_km=0.275,
        max_i_ka=0.265,
        c_nf_per_km=0
    )

    pp.create_line_from_parameters(
        net=net,
        from_bus=bus1,
        to_bus=bus2,
        length_km=1.4,
        r_ohm_per_km=0.33,
        x_ohm_per_km=0.234,
        max_i_ka=0.33,
        c_nf_per_km=0
    )

    pp.create_line_from_parameters(
        net=net,
        from_bus=bus8,
        to_bus=bus7,
        length_km=1.1,
        r_ohm_per_km=0.12,
        x_ohm_per_km=0.378,
        max_i_ka=0.605,
        c_nf_per_km=0
    )

    pp.create_line_from_parameters(
        net=net,
        from_bus=bus5,
        to_bus=bus6,
        length_km=1.3,
        r_ohm_per_km=0.157,
        x_ohm_per_km=0.377,
        max_i_ka=0.51,
        c_nf_per_km=0
    )

    pp.create_line_from_parameters(
        net=net,
        from_bus=bus0,
        to_bus=bus1,
        length_km=1.2,
        r_ohm_per_km=0.33,
        x_ohm_per_km=0.234,
        max_i_ka=0.33,
        c_nf_per_km=0
    )

    # Создание коммутационных аппаратов (switches)
    pp.create_switch(
        net=net,
        bus=bus2,
        element=bus3,
        et="b",
        closed=True
    )

    pp.create_switch(
        net=net,
        bus=bus4,
        element=bus5,
        et="b",
        closed=True
    )

    pp.create_switch(
        net=net,
        bus=bus7,
        element=bus6,
        et="b",
        closed=True
    )

    # Создание трансформаторов
    pp.create_transformer_from_parameters(
        net=net,
        hv_bus=bus08,
        lv_bus=bus8,
        sn_mva=32,
        vn_hv_kv=110,
        vn_lv_kv=10,
        vk_percent=10.5,
        i0_percent=0.28,
        pfe_kw=160,
        vkr_percent=(160 / 320)
    )

    pp.create_transformer_from_parameters(
        net=net,
        hv_bus=bus35,
        lv_bus=bus5,
        sn_mva=40,
        vn_hv_kv=110,
        vn_lv_kv=10,
        vk_percent=11,
        i0_percent=0.3,
        pfe_kw=170,
        vkr_percent=(170 / 400)
    )

    pp.create_transformer_from_parameters(
        net=net,
        hv_bus=bus34,
        lv_bus=bus4,
        sn_mva=40,
        vn_hv_kv=110,
        vn_lv_kv=10,
        vk_percent=11,
        i0_percent=0.3,
        pfe_kw=170,
        vkr_percent=(170 / 400)
    )

    return net

In [5]:
net = create_network("net")
pp.runpp(net)

In [6]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.020000,0.000000,-81.680006,-69.214793
1,1.016177,0.017814,12.000000,11.500000
2,1.012474,0.032392,23.000000,21.500000
3,1.012474,0.032392,27.000000,22.000000
4,0.999099,-0.872649,5.000000,3.000000
5,0.999099,-0.872649,3.000000,2.000000
6,0.981244,-1.686363,4.000000,3.000000
7,0.981244,-1.686363,2.500000,1.600000
8,0.995999,-1.199105,4.000000,3.500000
9,1.017526,0.028574,0.000000,0.000000


In [7]:
net.shunt

,bus,name,q_mvar,p_mw,vn_kv,step,max_step,id_characteristic_table,step_dependency_table,in_service


In [8]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,7.033467,6.436801,-7.013206,-6.424688,0.020262,0.012113,0.049061,0.049061,0.049061,1.020000,0.000000,1.017526,0.028574,18.513457
1,6.046829,3.650958,-6.029066,-3.640339,0.017763,0.010619,0.036617,0.036617,0.036617,1.012474,0.032392,1.009507,0.033128,13.817826
2,6.060305,3.744546,-6.043931,-3.734757,0.016374,0.009789,0.036930,0.036930,0.036930,1.012474,0.032392,1.009763,0.034644,13.935756
3,62.347288,51.065796,-62.107134,-50.895504,0.240154,0.170291,0.416259,0.416259,0.416259,1.016177,0.017814,1.012474,0.032392,126.139107
4,2.837576,2.641955,-2.817575,-2.578950,0.020002,0.063005,0.224743,0.224743,0.224743,0.995999,-1.199105,0.981244,-1.686363,37.147546
5,3.719829,2.110866,-3.682425,-2.021050,0.037403,0.089815,0.247156,0.247156,0.247156,0.999099,-0.872649,0.981244,-1.686363,48.462036
6,74.646539,62.777992,-74.347288,-62.565796,0.299251,0.212196,0.501891,0.501891,0.501891,1.020000,0.000000,1.016177,0.017814,152.088185


#### Рассчитываем потери на линиях при мощности КУ 0%

In [9]:
# Рассчитываем потери на линиях при мощности КУ 0%

# Со стороны высокого напряжения
line_pw_0percent_110 = net.res_line.pl_mw
line_qw_0percent_110 = net.res_line.ql_mvar

Sum_line_pw_0percent_110 = (
    sum(line_pw_0percent_110[0:4]) +
    sum(line_pw_0percent_110[6:7])
)

Sum_line_qw_0percent_110 = (
    sum(line_qw_0percent_110[0:4]) +
    sum(line_qw_0percent_110[6:7])
)

Sum_line_all_0percent_110 = math.sqrt(
    pow(Sum_line_pw_0percent_110, 2) +
    pow(Sum_line_qw_0percent_110, 2)
)

# Со стороны низкого напряжения
line_pw_0percent_10 = net.res_line.pl_mw
line_qw_0percent_10 = net.res_line.ql_mvar

Sum_line_pw_0percent_10 = sum(line_pw_0percent_10[4:6])
Sum_line_qw_0percent_10 = sum(line_qw_0percent_10[4:6])

Sum_line_all_0percent_10 = math.sqrt(
    pow(Sum_line_pw_0percent_10, 2) +
    pow(Sum_line_qw_0percent_10, 2)
)

# Суммарные потери
Sum_line_all_0percent = (
    Sum_line_all_0percent_110 +
    Sum_line_all_0percent_10
)

print("Потери на линиях при КУ = 0%:")
print(f"  ВН (110 кВ): {Sum_line_all_0percent_110:.4f} МВА")
print(f"  НН (10 кВ):  {Sum_line_all_0percent_10:.4f} МВА")
print(f"  Всего:       {Sum_line_all_0percent:.4f} МВА")

Потери на линиях при КУ = 0%:
  ВН (110 кВ): 0.7245 МВА
  НН (10 кВ):  0.1632 МВА
  Всего:       0.8877 МВА


#### Рассчитываем потери на линиях при мощности КУ 25%

In [10]:
# Мощность 25%
net25 = create_network("net25")

pp.create_shunt_as_capacitor(
    net25,
    bus=bus4,
    q_mvar=3 * 0.25,
    loss_factor=0,
    in_service=True
)

pp.create_shunt_as_capacitor(
    net25,
    bus=bus5,
    q_mvar=1 * 0.25,
    loss_factor=0,
    in_service=True
)

pp.create_shunt_as_capacitor(
    net25,
    bus=bus7,
    q_mvar=1.1 * 0.25,
    loss_factor=0,
    in_service=True
)

pp.runpp(net25)

NameError: name 'bus4' is not defined

In [ ]:
net25.shunt

,bus,name,q_mvar,p_mw,vn_kv,step,max_step,id_characteristic_table,step_dependency_table,in_service
0,4,None,-0.750,0.0,10.0,1,1,<NA>,False,True
1,5,None,-0.250,0.0,10.0,1,1,<NA>,False,True
2,7,None,-0.275,0.0,10.0,1,1,<NA>,False,True


In [ ]:
# Рассчитываем потери на линиях при мощности КУ 25%

# Со стороны высокого напряжения
line_pw_25percent_110 = net25.res_line.pl_mw
line_qw_25percent_110 = net25.res_line.ql_mvar

Sum_line_pw_25percent_110 = (
    sum(line_pw_25percent_110[0:4]) +
    sum(line_pw_25percent_110[6:7])
)

Sum_line_qw_25percent_110 = (
    sum(line_qw_25percent_110[0:4]) +
    sum(line_qw_25percent_110[6:7])
)

Sum_line_all_25percent_110 = math.sqrt(
    pow(Sum_line_pw_25percent_110, 2) +
    pow(Sum_line_qw_25percent_110, 2)
)

# Со стороны низкого напряжения
line_pw_25percent_10 = net25.res_line.pl_mw
line_qw_25percent_10 = net25.res_line.ql_mvar

Sum_line_pw_25percent_10 = sum(
    line_pw_25percent_10[4:6]
)

Sum_line_qw_25percent_10 = sum(
    line_qw_25percent_10[4:6]
)

Sum_line_all_25percent_10 = math.sqrt(
    pow(Sum_line_pw_25percent_10, 2) +
    pow(Sum_line_qw_25percent_10, 2)
)

# Суммарные потери
Sum_line_all_25percent = (
    Sum_line_all_25percent_110 +
    Sum_line_all_25percent_10
)

print("Потери на линиях при КУ = 25%:")
print(f"  ВН (110 кВ): {Sum_line_all_25percent_110:.4f} МВА")
print(f"  НН (10 кВ):  {Sum_line_all_25percent_10:.4f} МВА")
print(f"  Всего:       {Sum_line_all_25percent:.4f} МВА")

Потери на линиях при КУ = 25%:


  ВН (110 кВ): 0.7107 МВА
  НН (10 кВ):  0.1568 МВА
  Всего:       0.8675 МВА


#### Рассчитываем потери на линиях при мощности КУ 50%

In [ ]:
# Мощность 50%

net50 = create_network("net50")

pp.create_shunt_as_capacitor(
    net50,
    bus=bus4,
    q_mvar=3 * 0.5,
    loss_factor=0,
    in_servise=True
)
pp.create_shunt_as_capacitor(
    net50,
    bus=bus5,
    q_mvar=1 * 0.5,
    loss_factor=0,
    in_servise=True
)
pp.create_shunt_as_capacitor(
    net50,
    bus=bus7,
    q_mvar=1.1 * 0.5,
    loss_factor=0,
    in_servise=True
)
pp.runpp(net50)

In [ ]:
net50.shunt

,bus,name,q_mvar,p_mw,vn_kv,step,max_step,id_characteristic_table,step_dependency_table,in_service,in_servise
0,4,None,-1.50,0.0,10.0,1,1,<NA>,False,True,True
1,5,None,-0.50,0.0,10.0,1,1,<NA>,False,True,True
2,7,None,-0.55,0.0,10.0,1,1,<NA>,False,True,True


In [ ]:
# Рассчитываем потери на линиях при мощности КУ 50%

# Со стороны высокого напряжения
line_pw_50percent_110 = net50.res_line.pl_mw
line_qw_50percent_110 = net50.res_line.ql_mvar

Sum_line_pw_50percent_110 = (
    sum(line_pw_50percent_110[0:4]) +
    sum(line_pw_50percent_110[6:7])
)

Sum_line_qw_50percent_110 = (
    sum(line_qw_50percent_110[0:4]) +
    sum(line_qw_50percent_110[6:7])
)

Sum_line_all_50percent_110 = math.sqrt(
    pow(Sum_line_pw_50percent_110, 2) +
    pow(Sum_line_qw_50percent_110, 2)
)

# Со стороны низкого напряжения
line_pw_50percent_10 = net50.res_line.pl_mw
line_qw_50percent_10 = net50.res_line.ql_mvar

Sum_line_pw_50percent_10 = sum(line_pw_50percent_10[4:6])
Sum_line_qw_50percent_10 = sum(line_qw_50percent_10[4:6])

Sum_line_all_50percent_10 = math.sqrt(
    pow(Sum_line_pw_50percent_10, 2) +
    pow(Sum_line_qw_50percent_10, 2)
)

# Суммарные потери
Sum_line_all_50percent = (
    Sum_line_all_50percent_110 +
    Sum_line_all_50percent_10
)

print("Потери на линиях при КУ = 50%:")
print(f"  ВН (110 кВ): {Sum_line_all_50percent_110:.4f} МВА")
print(f"  НН (10 кВ):  {Sum_line_all_50percent_10:.4f} МВА")
print(f"  Всего:       {Sum_line_all_50percent:.4f} МВА")

Потери на линиях при КУ = 50%:
  ВН (110 кВ): 0.6976 МВА
  НН (10 кВ):  0.1508 МВА
  Всего:       0.8484 МВА


#### Рассчитываем потери на линиях при мощности КУ 75%

In [ ]:
# Мощность 75%

net75 = create_network("net75")

pp.create_shunt_as_capacitor(
    net=net75,
    bus=bus4,
    q_mvar=3 * 0.75,
    loss_factor=0,
    in_servise=True
)
pp.create_shunt_as_capacitor(
    net=net75,
    bus=bus5,
    q_mvar=1 * 0.75,
    loss_factor=0,
    in_servise=True
)
pp.create_shunt_as_capacitor(
    net=net75,
    bus=bus7,
    q_mvar=1.1 * 0.75,
    loss_factor=0,
    in_servise=True
)

pp.runpp(net75)

In [ ]:
net75.shunt

,bus,name,q_mvar,p_mw,vn_kv,step,max_step,id_characteristic_table,step_dependency_table,in_service,in_servise
0,4,None,-2.250,0.0,10.0,1,1,<NA>,False,True,True
1,5,None,-0.750,0.0,10.0,1,1,<NA>,False,True,True
2,7,None,-0.825,0.0,10.0,1,1,<NA>,False,True,True


In [ ]:
# Рассчитываем потери на линиях при мощности КУ 75%
# Со стороны высокого напряжения
line_pw_75percent_110 = net75.res_line.pl_mw
line_qw_75percent_110 = net75.res_line.ql_mvar
Sum_line_pw_75percent_110 = (
    sum(line_pw_75percent_110[0:4]) +
    sum(line_pw_75percent_110[6:7])
)
Sum_line_qw_75percent_110 = (
    sum(line_qw_75percent_110[0:4]) +
    sum(line_qw_75percent_110[6:7])
)
Sum_line_all_75percent_110 = math.sqrt(
    pow(Sum_line_pw_75percent_110, 2) +
    pow(Sum_line_qw_75percent_110, 2)
)
# Со стороны низкого напряжения
line_pw_75percent_10 = net75.res_line.pl_mw
line_qw_75percent_10 = net75.res_line.ql_mvar
Sum_line_pw_75percent_10 = sum(
    line_pw_75percent_10[4:6]
)
Sum_line_qw_75percent_10 = sum(
    line_qw_75percent_10[4:6]
)
Sum_line_all_75percent_10 = math.sqrt(
    pow(Sum_line_pw_75percent_10, 2) +
    pow(Sum_line_qw_75percent_10, 2)
)
# Суммарные потери
Sum_line_all_75percent = (
    Sum_line_all_75percent_110 +
    Sum_line_all_75percent_10
)
print("Потери на линиях при КУ = 75%:")
print(f"  ВН (110 кВ): {Sum_line_all_75percent_110:.4f} МВА")
print(f"  НН (10 кВ):  {Sum_line_all_75percent_10:.4f} МВА")
print(f"  Всего:       {Sum_line_all_75percent:.4f} МВА")

Потери на линиях при КУ = 75%:
  ВН (110 кВ): 0.6851 МВА
  НН (10 кВ):  0.1454 МВА
  Всего:       0.8304 МВА


#### Рассчитываем потери на линиях при мощности КУ 100%

In [ ]:
# Мощность 100%

net100 = create_network("net100")

pp.create_shunt_as_capacitor(
    net=net100,
    bus=bus4,
    q_mvar=3 * 1.00,
    loss_factor=0,
    in_servise=True
)
pp.create_shunt_as_capacitor(
    net=net100,
    bus=bus5,
    q_mvar=1 * 1.00,
    loss_factor=0,
    in_servise=True
)
pp.create_shunt_as_capacitor(
    net=net100,
    bus=bus7,
    q_mvar=1.1 * 1.00,
    loss_factor=0,
    in_servise=True
)

pp.runpp(net100)

In [ ]:
net100.shunt

,bus,name,q_mvar,p_mw,vn_kv,step,max_step,id_characteristic_table,step_dependency_table,in_service,in_servise
0,4,None,-3.0,0.0,10.0,1,1,<NA>,False,True,True
1,5,None,-1.0,0.0,10.0,1,1,<NA>,False,True,True
2,7,None,-1.1,0.0,10.0,1,1,<NA>,False,True,True


In [ ]:
# Рассчитываем потери на линиях при мощности КУ 100%
# Со стороны высокого напряжения
line_pw_100percent_110 = net100.res_line.pl_mw
line_qw_100percent_110 = net100.res_line.ql_mvar
Sum_line_pw_100percent_110 = (
    sum(line_pw_100percent_110[0:4]) +
    sum(line_pw_100percent_110[6:7])
)
Sum_line_qw_100percent_110 = (
    sum(line_qw_100percent_110[0:4]) +
    sum(line_qw_100percent_110[6:7])
)
Sum_line_all_100percent_110 = math.sqrt(
    pow(Sum_line_pw_100percent_110, 2) +
    pow(Sum_line_qw_100percent_110, 2)
)
# Со стороны низкого напряжения
line_pw_100percent_10 = net100.res_line.pl_mw
line_qw_100percent_10 = net100.res_line.ql_mvar
Sum_line_pw_100percent_10 = sum(
    line_pw_100percent_10[4:6]
)
Sum_line_qw_100percent_10 = sum(
    line_qw_100percent_10[4:6]
)
Sum_line_all_100percent_10 = math.sqrt(
    pow(Sum_line_pw_100percent_10, 2) +
    pow(Sum_line_qw_100percent_10, 2)
)
# Суммарные потери
Sum_line_all_100percent = (
    Sum_line_all_100percent_110 +
    Sum_line_all_100percent_10
)
print("Потери на линиях при КУ = 100%:")
print(f"  ВН (110 кВ): {Sum_line_all_100percent_110:.4f} МВА")
print(f"  НН (10 кВ):  {Sum_line_all_100percent_10:.4f} МВА")
print(f"  Всего:       {Sum_line_all_100percent:.4f} МВА")

Потери на линиях при КУ = 100%:
  ВН (110 кВ): 0.6732 МВА
  НН (10 кВ):  0.1404 МВА
  Всего:       0.8136 МВА
